In [5]:
import requests
import urllib.request
import pandas as pd
from pandas.io.html import read_html
import bs4
from bs4 import BeautifulSoup
import numpy as np

import re
import lxml

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

# https://gist.github.com/aculich/b34868c098d94d614515
# https://beenje.github.io/blog/posts/parsing-html-tables-in-python-with-pandas/
# https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/

[nltk_data] Downloading package punkt to
[nltk_data]     /home/cabbagepatch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cabbagepatch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/cabbagepatch/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### Start by importing data from wikipedia.

In [2]:
billboard = "https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_chart_achievements_by_decade"
page = requests.get(billboard)
soup = BeautifulSoup(page.content, 'html.parser')
table_classes = {"class": ["wikitable", "plainrowheaders"]}
wikitables = soup.findAll("table", table_classes)


NameError: name 'requests' is not defined

In [6]:
billboard = "https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_chart_achievements_by_decade"
wikitables = read_html(billboard, flavor='bs4', index_col=0, header=0, attrs={"class":"wikitable"})

print("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 25 wikitables


In [7]:
# here, checked all tables by hand to decide how to clean.
wikitables[24]

,Artist,Weeks atnumber one
Song,,
"""Mack the Knife""",Bobby Darin,9.0
"""Theme from A Summer Place""",Percy Faith,NaN
"""Hey Jude""",The Beatles,NaN
"""Tossin' and Turnin'""",Bobby Lewis,7.0
"""I Want to Hold Your Hand""",The Beatles,NaN
"""I'm a Believer""",The Monkees,NaN
"""I Heard It Through the Grapevine""",Marvin Gaye,NaN
"""It's All in the Game""",Tommy Edwards,6.0
"""The Battle of New Orleans""",Johnny Horton,NaN


### Now we start cleaning the data, beginning with collecting all artist names.

In [8]:
# splitting wikitables into smaller dfs for cleaning.

# collecting artist names in index column
pd1 = pd.concat(wikitables[0:2], axis=0, sort=False)
pd2 = pd.concat(wikitables[4:6], axis=0, sort=False)
pd3 = pd.concat(wikitables[9:11], axis=0, sort=False)
pd4 = pd.concat(wikitables[13:15], axis=0, sort=False)
pd5 = pd.concat(wikitables[17:19], axis=0, sort=False)
pd6 = pd.concat(wikitables[21:23], axis=0, sort=False)

pdx = [pd1, pd2, pd3, pd4, pd5, pd6]
pdx = pd.concat(pdx[:], axis=0, sort=False)
df1 = pdx.index.values

print("df1:", len(df1), "names")

df1: 128 names


In [9]:
# collecting artist names in col 1

pd1 = pd.concat(wikitables[2:4], axis=0, sort=False)
pd2 = wikitables[6]
pd3 = wikitables[8]
pd4 = wikitables[12]
pd5 = wikitables[16]
pd6 = wikitables[20]
pd7 = wikitables[24]

pd_why = [pd1, pd2, pd3, pd4, pd5, pd6, pd7]
pd_why = pd.concat(pd_why, axis=0, sort=False)
df2 = pd_why['Artist'].values

print("df2:", len(df2), "names")

df2: 108 names


In [10]:
pd1 = wikitables[15]
pd2 = wikitables[19]
pd3 = wikitables[23]
pd4 = wikitables[7]
pd5 = wikitables[11]

pd_zzz = [pd1, pd2, pd3, pd4, pd5]
pd_zzz = pd.concat(pd_zzz, axis=0, sort=False)
df3 = pd_zzz['Artist'].values

print("df3:", len(df3), "names")

df3: 50 names


In [11]:
df = np.array([])
df = np.concatenate((df1, df2, df3))
pos_names = pd.DataFrame(data=df, columns=['Artists'])
pos_names = pos_names['Artists'].unique()

pos_names

array(['Rihanna', 'Katy Perry', 'Bruno Mars', 'Drake', 'Justin Bieber',
       'Taylor Swift', 'Adele', 'Eminem', 'Kesha', 'The Weeknd',
       'Maroon 5', 'Cardi B', 'Post Malone', 'Pharrell Williams',
       'Ed Sheeran', 'Usher', 'Beyoncé', 'Nelly', 'Ludacris',
       'Justin Timberlake', 'Mariah Carey', '50 Cent', 'Alicia Keys',
       'The Black Eyed Peas', 'Jay-Z', 'Janet Jackson', 'Boyz II Men',
       'Madonna', 'Whitney Houston', 'Celine Dion', 'TLC',
       'Wilson Phillips', 'Paula Abdul', 'Bryan Adams', 'Puff Daddy',
       'Monica', 'Céline Dion', 'Elton John', 'Brandy', 'Michael Jackson',
       'Phil Collins', 'George Michael', 'Daryl Hall & John Oates',
       'Lionel Richie', 'Stevie Wonder', 'Bon Jovi', 'Prince',
       'Paul McCartney', 'Olivia Newton-John', 'Diana Ross', 'Bee Gees',
       'Paul McCartney and Wings', 'Eagles', 'The Jackson 5',
       'John Denver', 'KC & The Sunshine Band', 'Barbra Streisand',
       'Donna Summer', 'Rod Stewart', 'Andy Gibb', 'Robe

In [12]:
# trying to remove rows with "and" and "featuring"

pnames = pos_names[~pos_names.Artists.str.contains(stop_words)]
pnames

AttributeError: 'numpy.ndarray' object has no attribute 'Artists'

### These artists are our positive dataset (control variable), who are are alive, happy, and healthy. 
### Now we can begin analyzing lyrics from this pool of artists.

In [13]:
songsTable1 = pd.read_csv("lyrics/lyrics1.csv")
songsTable1.head()

,Band,Lyrics,Song
0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday
1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die
2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside
3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot
4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds


In [14]:
songsTable2 = pd.read_csv("lyrics/lyrics2.csv")
songsTable2.head()

,Band,Lyrics,Song
0,Adam Gregory,I was born in the country\r\nFar away from cit...,In the Country
1,Adam Gregory,Do you feel your heart beating a million miles...,Me Too
2,Adam Gregory,There are moments in your life \r\nThat are fr...,Memory Like That
3,Adam Gregory,There's a town by a river\r\nAnd a dirt road t...,Sweet Memories
4,Adam Gregory,I go walkin' down a long road tonight\r\nI go ...,Walkin'
